# Walmart API overview
##### Search API used for look ups. We will use this first
##### Taxonomy API used for classifications.  We want to find a way to count classifications

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint

# We are using 2 api Keys for redundancy and because
# there is a limit to the number of calls
api_key = "4tsbjxvbrwnpcjdsh277csqu"
api_alt = "anrtcd5dheh9758z5wgdy677"

# Base url for all API look ups
base_url = "http://api.walmartlabs.com/v1/" 

# Variables to specify the API
search = "search?"
paginated_products = "paginated/items?"

# Base url for search
search_url = base_url + search
paginated_url = base_url + paginated_products



# Search API

In [130]:
# Load search criteria for Walmart Search API.
# reference URL: https://developer.walmartlabs.com/docs/read/Search_API

format = "json"
categoryId = "3944"
query = "laptop"
numItems = 25 # max 25. 10 is default
start = 2
sort = "price"
order = "desc"
facet = "on" # boolean
min = str(0) # lower end of range for search within price range
max = str(200) # higher end of range price for search with price range
facet_range = (f"price:[{min}%20TO%20{max}]") # note the format [num1:num2] does not work. Use %20TO%20 for ":"



# NOTE: we are using 2 methods to create urls. Because the params argument does not format the output
# in the manner needed to get the API to work.  This is used only for price range lookups

# manual url. Used for price range lookup only
manual_url = (f"{url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")

print(manual_url)

# API parameters for other lookups
params = {
    "apiKey": api_alt,
    #"start": start, # note keep turned off.  We can't get this to work yet
    "format": "json", # json|xml
    "categoryId": categoryId,
    "query": query,
    "numItems": numItems,
    #"sort": sort, #allowed sort types are [relevance, price, title, bestseller, customerRating, new]
    #"order": order,  #allowed values are [asc, desc]
    #"facet":facet, # note keep turned off.  We can't get this to work yet
    #"facet.range":facet_range    # note keep turned off.  We can't get this to work yet
}

http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[0%20TO%20200]


In [131]:
# We are going to run through this one time to make sure we are pulling data correctly
# There is a limitation of 25 results per search.  
# Bellow we will create a loop to pull multiple pages
# We will try two methods: price range and paginated results (has a limit of 1000 items)
# Remember we are limited to 5000 requests per day

walmart_data = requests.get(manual_url) # or requests.get(search_url, params=params)
walmart_url = walmart_data.url
walmart_data = walmart_data.json()
print(walmart_url)

numItems = walmart_data["numItems"]

#create dictionary to store walmart_data
result = {}
result["totalResults"] = walmart_data["totalResults"]
result["start"] = walmart_data["start"]
result["numItems"] = walmart_data["numItems"]
result["items"] = walmart_data["items"]



http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[0%20TO%20200]
{'query': 'laptop', 'categoryId': '3944', 'sort': 'relevance', 'responseGroup': 'base', 'totalResults': 3089, 'start': 1, 'numItems': 25, 'items': [{'itemId': 54056482, 'parentItemId': 54056482, 'name': 'HP Stream 14" Laptop, Windows 10 Home, Office 365 Personal 1-year included, Intel Celeron N3060 Processor, 4GB RAM, 32GB eMMC Storage', 'msrp': 219.0, 'salePrice': 199.86, 'upc': '190780429624', 'categoryPath': 'Electronics/Computers/Laptops/Shop Laptops by Type/All Laptop Computers', 'shortDescription': 'Get the essential productivity you want, without slowing down. This nimble laptop lets you easily shift from homework workhorse to marathoning your favorite shows. Office 365 Personal 1-year subscription included.', 'longDescription': "&lt;br&gt;&lt;b&gt;HP Stream 14&quot; Laptop:&lt;/b&gt;&lt;br&gt;&lt;br&gt;&lt;b&gt;Key F

TypeError: 'NoneType' object is not subscriptable

In [132]:
print (result["items"][24]['itemId'])
print (walmart_data['totalResults'])

470010299
3089


In [100]:
print(params)
print (result)
print(json.dumps(walmart_data, indent=4))

{'apiKey': 'anrtcd5dheh9758z5wgdy677', 'format': 'json', 'categoryId': '3944', 'query': 'laptop', 'numItems': 25}
{'totalResults': 1096, 'start': 1, 'numItems': 25, 'items': [{'itemId': 51023513, 'parentItemId': 51023513, 'name': 'HP ZBook 17 G3 Mobile Workstation - 17.3" - Xeon E3-1535MV5 - 16 GB RAM - 512 GB SSD + 1 TB HDD', 'msrp': 7755.99, 'salePrice': 3715.71, 'upc': '889899422380', 'categoryPath': 'Electronics/Computers/Laptops/Laptops By Brand/HP Laptops', 'shortDescription': '&quot;Xeon E3-1535MV5 / 2.9 GHz - Win 7 Pro 64-bit (includes Win 10 Pro 64-bit License) - 16 GB RAM - 512 GB SSD HP Z Turbo Drive G2 + 1 TB HDD - 17.3&quot;&quot; IPS 1920 x 1080 (Full HD) - Quadro M5000M / HD Graphics P530 - Wi-Fi Blu&quot;', 'longDescription': 'HP ZBook 17 G3 Mobile Workstation - 17.3&quot; - Xeon E3-1535MV5 - 16 GB RAM - 512 GB SSD + 1 TB HDD', 'thumbnailImage': 'https://i5.walmartimages.com/asr/48fdaafb-3b23-48e4-a24c-2467d53a8ab1_1.6f8d67b4702377c9b023e9f0f69a98f1.jpeg?odnHeight=100&o

In [161]:
print (itemId_list)

[586976837]


In [101]:
# Use for loop Create data series from the ['items'][0] json fields
# Method 1 use append

# create empty data series to populate
itemId_list = []
name_list = []
categoryPath_list = []
salePrice_list = []
upc_list = []
customerRating_list = []
numReviews_list = []
stock_list = []
offerType_list = []
availableOnline_list = []

for i  in range (len(walmart_data['items'])):
    #if a field is blank, it will create a KeyError. Use exception to skip
    try:
        itemId_list.append(walmart_data['items'][i]['itemId'])
        name_list.append(walmart_data['items'][i]['name'])
        categoryPath_list.append(walmart_data['items'][i]['categoryPath'])
        salePrice_list.append(walmart_data['items'][i]['salePrice'])
        upc_list.append(walmart_data['items'][i]['upc'])
        customerRating_list.append(walmart_data['items'][i]['customerRating'])
        numReviews_list.append(walmart_data['items'][i]['numReviews'])
        stock_list.append(walmart_data['items'][i]['stock'])
        offerType_list.append(walmart_data['items'][i]['offerType'])
        availableOnline_list.append(walmart_data['items'][i]['availableOnline'])
    except KeyError:
        print (f"Skipped line {i}")

# test output by printing
print (itemId_list)
print (name_list)
print (salePrice_list)
print (upc_list)
print (customerRating_list)
print (numReviews_list)
print (stock_list)
print (offerType_list)
print (availableOnline_list)
print (categoryPath_list)

Skipped line 0
Skipped line 1
Skipped line 2
Skipped line 3
Skipped line 4
Skipped line 5
Skipped line 6
Skipped line 8
Skipped line 9
Skipped line 10
Skipped line 11
Skipped line 12
Skipped line 13
Skipped line 14
Skipped line 15
Skipped line 16
Skipped line 17
Skipped line 18
Skipped line 19
Skipped line 20
Skipped line 21
Skipped line 22
Skipped line 23
Skipped line 24
[51023513, 654068086, 605539612, 254392836, 900056702, 747880448, 151901135, 51894007, 210958136, 265694594, 135595307, 913134696, 622480476, 686450764, 407161199, 455547550, 261991124, 269193997, 223358894, 492768400, 195335306, 264048810, 383651934, 188122804, 272846488]
['HP ZBook 17 G3 Mobile Workstation - 17.3" - Xeon E3-1535MV5 - 16 GB RAM - 512 GB SSD + 1 TB HDD', 'MSI WT73VR 7RM-1211US 17.3" Workstation Laptop i7-7820HK 64GB 512GB 1TB P5000', 'MSI GT75 TITAN 4K-071 VR 17.3" Gaming Laptop i9-8950HK 32GB 1TB 1TB GTX 1080 8GB', 'Lenovo ThinkPad P50 Mobile Workstation Laptop - Windows 10 Pro - Intel i7-6820HQ, 64G

In [162]:
print(len(itemId_list))

1


In [102]:
# create output file
with open('Walmart_datasets/walmart_laptops.json', 'w') as outfile:
    json.dump(walmart_data, outfile)

In [103]:
# loading the saved file in order to perform additional manipulation
# this is just a test for now

from pprint import pprint
import os
filepath = os.path.join('Walmart_datasets/walmart_laptops.json')
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
    print (json_data['items'][0]['name'])

HP ZBook 17 G3 Mobile Workstation - 17.3" - Xeon E3-1535MV5 - 16 GB RAM - 512 GB SSD + 1 TB HDD


In [165]:
# Loop by pricing parameters to pull out entire data set for a product category
# to save time we are going to hard code some numbers then go back later if we have time

# Number of items found in our search
# there are 20,747 total results for laptop in electronics
# step 0 there are 3089 results in the price range $0-$200
# step 1 there are 3015 results in the price range $201-$400
# step 2 there are 2053 results in the price range $401-$600
# step 3 there are 2995 results in the price range $601-$1000
# step 4 there are 2021 results in the price range $1001-$1400
# step 5 there are 2726 results in the price range $1401-$1800
# step 6 there are 1873 results in the price range $1801-$2200
# step 7 there are 1011 results in the price range $2201-$2600
# step 8 there are 863 results in the price range $2601-3000
# step 9 there are 1096 results in the price range above $3000

category_total = 20747 #hard coded from broad search
step0 = 200*25//3089
step1 = 200*25//3015
step2 = 200*25//2053
step3 = 400*25//2995
step4 = 400*25//2021
step5 = 400*25//2726
step6 = 400*25//1873
step7 = 400*25//1011
step8 = 400*25//863

# create a function to perform the price search

# reset the list values to 0
itemId_list = []
name_list = []
categoryPath_list = []
salePrice_list = []
upc_list = []
customerRating_list = []
numReviews_list = []
stock_list = []
offerType_list = []
availableOnline_list = []


def price_data(lowRange, highRange):
    facet_range = (f"price:[{lowRange}%20TO%20{highRange}]")
    price_url = (f"{url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")
    walmart_data2 = requests.get(price_url)
    walmart_data2 = walmart_data2.json()
    
    for i  in range (len(walmart_data2['items'])):
        #if a field is blank, it will create a KeyError. Use exception to skip
        try:
            itemId_list.append(walmart_data2['items'][i]['itemId'])
            name_list.append(walmart_data2['items'][i]['name'])
            categoryPath_list.append(walmart_data2['items'][i]['categoryPath'])
            salePrice_list.append(walmart_data2['items'][i]['salePrice'])
            upc_list.append(walmart_data2['items'][i]['upc'])
            customerRating_list.append(walmart_data2['items'][i]['customerRating'])
            numReviews_list.append(walmart_data2['items'][i]['numReviews'])
            stock_list.append(walmart_data2['items'][i]['stock'])
            offerType_list.append(walmart_data2['items'][i]['offerType'])
            availableOnline_list.append(walmart_data2['items'][i]['availableOnline'])
        except KeyError:
            print (f"Skipped line {i}")
            
    print(name_list)
    


In [154]:
price_data(0,2)

{'query': 'laptop',
 'categoryId': '3944',
 'sort': 'relevance',
 'responseGroup': 'base',
 'totalResults': 1,
 'start': 1,
 'numItems': 1,
 'items': [{'itemId': 586976837,
   'parentItemId': 586976837,
   'name': 'Msi Gl63 I7-8750/gtx1050ti/8gb/256g+1t/win10',
   'salePrice': 1.0,
   'upc': '824142162644',
   'categoryPath': 'Electronics/Computers/Laptops/Shop Laptops by Type/Gaming Laptops',
   'shortDescription': 'short description is not available',
   'longDescription': 'Msi Gl63 I7-8750/gtx1050ti/8gb/256g+1t/win10',
   'thumbnailImage': 'https://i5.walmartimages.com/asr/0eaab542-0ae6-47ca-9c1d-cf2d8832bfe2_1.49c26fecf52964e49c0ed3c067f470ab.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF',
   'mediumImage': 'https://i5.walmartimages.com/asr/0eaab542-0ae6-47ca-9c1d-cf2d8832bfe2_1.49c26fecf52964e49c0ed3c067f470ab.jpeg?odnHeight=180&odnWidth=180&odnBg=FFFFFF',
   'largeImage': 'https://i5.walmartimages.com/asr/0eaab542-0ae6-47ca-9c1d-cf2d8832bfe2_1.49c26fecf52964e49c0ed3c067f470ab.jpeg

In [166]:
# Use for loop to go through prices in the steps defined

itemCount = 1
low = 200
high = step0

for itemCount in range (1,2):
    if high < 2000:
        print(low)
        high = low + step0
        print(high)
        price_data(low,high)
        low = high
        print(low)
        itemCount += 1
        print(itemCount)


200
201
Skipped line 0
Skipped line 2
Skipped line 3
Skipped line 4
Skipped line 5
Skipped line 6
Skipped line 7
Skipped line 8
Skipped line 9
Skipped line 10
Skipped line 11
Skipped line 12
['Refurbished ASUS TP401NA-WH21T 14" Touchscreen Laptop Quad-Core 2.5GHz 4GB RAM 64GB EMMC', 'Refurbished Dell Latitude E5430 i5 2.7GHz 8GB 128SSD DVD Win 10 Pro 64 Laptop Computer B', 'Refurbished Dell Latitude E5440 2.0GHz i5 8GB 320GB DVD Windows 10 Pro 64 Laptop B', 'Refurbished Dell Latitude E5440 2.0GHz i5 4GB 320GB DRW Windows 10 Pro 64 Laptop Camera', "Refurbished Dell Silver Latitude E6410 14.1'' PC Laptop Intel i5 Dual Core 2.4GHz 4GB RAM 320GB HDD NVIDIA NVS 3100M 1440 x 900 Display Windows 10 Home 64-Bit", 'Refurbished Lenovo ThinkPad T520 Laptop  i5 Dual-Core 4GB 1TB Win 7 Pro 3 Yr Wty B v.WAA', 'Refurbished Lenovo ThinkPad X240 1.9GHz i5 4GB 128SSD Windows 10 Pro 64 12" Laptop B', 'Refurbished Dell Latitude E5440 1.9GHz i5 4GB 320GB DVD Windows 10 Pro 64 Laptop B', 'Refurbished Dell L